In [1]:
# This file is an implementation of the 2D LIPM
# Author : Avadesh Meduri
# Date : 21/04/2020

import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [174]:
'''
This is an implementation of a 2D Lipm environment with variable number of steps possible
'''

class TwoDLipmEnv:
    
    def __init__(self, h, b, max_step_length, w, no_actions = 9):
        '''
        Input:
            h : height of the lipm above the ground
            b : width of the base (distance between the feet)
            max_step_length : max step length allowed
            w : weights for the cost computation
            no_actions : number of discretizations
        '''
        
        self.omega = np.sqrt(9.81/h)
        self.max_leg_length = 0.3
        self.dt = 0.001
        self.h = h
        self.b = b
        self.no_steps = 0
        assert len(w) == 3
        self.w = w
        assert (np.linalg.norm([max_step_length, self.h]) < self.max_leg_length)
        # The co ordinate axis is x : forward and y : sideways walking, z : faces upward
        # This means that left leg is on the positive side of the y axis
        self.action_space_x = np.around(np.linspace(-max_step_length, max_step_length, no_actions), 2)
        # actions to the free side
        if b > 0 :
            self.action_space_ly = np.linspace(b, max_step_length + b, int(no_actions/2))
            # actions to the non free side where leg can hit the other leg
            # Y axis actions step length allowed such that robot can't step to the left of the left leg
            # or the right to the right leg (no criss crossing)
            self.action_space_ry = np.arange(0, b, self.action_space_ly[1] - self.action_space_ly[0])
            self.action_space_y = np.around(np.concatenate((self.action_space_ry, self.action_space_ly)), 2)
        else:
            self.action_space_y = np.around(np.linspace(0, max_step_length, int(no_actions/2)), 2)
            
        self.A = np.matrix([[1, 0, self.dt, 0], 
                            [0, 1, 0, self.dt], 
                            [self.dt*(self.omega**2), 0, 1, 0], 
                            [0, self.dt*(self.omega**2), 0, 1]])
        
        self.B = np.matrix([[0, 0], [0, 0], [-(self.omega**2)*self.dt, 0], [0, -(self.omega**2)*self.dt]])
        self.t = 0
        
    def integrate_lip_dynamics(self, x_t, u_t):
        '''
        integrates the dynamics of the lipm for one time step
        Input:
            x_t : current state of the lip ([x_com, y_com, xd_com, yd_com])
            u_t : current cop location ([u_x, u_y])
        '''
        assert np.shape(x_t) == (4,)
        x_t_1 = np.matmul(self.A, np.transpose(x_t)) + np.matmul(self.B, np.transpose(u_t))
        return x_t_1
    
    def reset_env(self, x0, epi_time):
        '''
        Resets environment for a new episode
        Input:
            x0 : initial state of the system
            epi_time : episode time
        '''
        assert np.shape(x0) == (4,)
        self.t = 0
        self.sim_data = np.zeros((8, int(epi_time/self.dt)+1))
        self.no_steps = 0
        assert (np.linalg.norm([x0[0], self.h]) < self.max_leg_length)
        assert (np.linalg.norm([x0[1], self.h]) < self.max_leg_length)
        self.sim_data[:,0][0:4] = x0
        self.sim_data[:,0][5] = -self.b/2
        self.sim_data[:,0][6] = self.h
        self.sim_data[:,0][7] = 1 # determines which leg is on the ground (1 is right leg)
        
        return np.take(self.sim_data[:,0], [0, 1, 2, 3, 7])

    def step_env(self, u, step_time):
        '''
        Integrates the dynamics of the lipm for the duration of a step (until next action is to be taken)
        Input:
            u : action
            step_time : the duration after which next step is taken
        '''
        for i in range(int(step_time/self.dt)):
            self.sim_data[:,self.t + 1][0:4] = self.integrate_lip_dynamics(self.sim_data[:,self.t][0:4], \
                                                                           self.sim_data[:,self.t][4:6])
            self.sim_data[:,self.t + 1][4:6] = self.sim_data[:,self.t][4:6] # u
            self.sim_data[:,self.t + 1][6] = self.sim_data[:,self.t][6] # h
            self.sim_data[:,self.t + 1][7] = self.sim_data[:,self.t][7] # n
            self.t += 1
        
        self.sim_data[:,self.t][4] += self.action_space_x[u[0]]
        self.sim_data[:,self.t][5] += self.sim_data[:,self.t][7]*self.action_space_y[u[1]]
        self.sim_data[:,self.t][7] = -1*self.sim_data[:,self.t][7]
        
        ## modifying state that is returned is such that the origin is u0 instead of the global origin
        ## This ensures that the state x[0] is bounded by the maximum leg size while collecting data
        processed_state = np.take(self.sim_data[:,self.t].copy(), [0, 1, 2, 3, 7]) 
        processed_state[0:2] -= self.sim_data[:,self.t][4:6]
        
        if not self.isdone():
            self.no_steps += 1
            
        return np.round(processed_state, 2), self.compute_cost(), self.isdone()
    
    def isdone(self):
        '''
        Checks if the kinematic constraints are violated
        '''
        tmp = np.linalg.norm(self.sim_data[:,self.t][0:2] - self.sim_data[:,self.t][4:6])
        current_leg_length = np.linalg.norm([tmp, self.h])
        if current_leg_length > self.max_leg_length:
            return True
        else:
            return False
    
    def compute_cost(self):
        '''
        Computes cost which is distance between the hip(closest hip depending on which foot is on the ground)
        and the foot + velocity of the center of mass + 1 if step length not equal to zero (after taking into
        account the offset) + 100 if episode terminates (kinematics constraints are violated)
        '''
        hip = self.sim_data[:,self.t][0:2].copy()
        hip[1] += -1*self.sim_data[:,self.t][7]*(self.b/2) # -1 is to match co ordinate axis
        u = self.sim_data[:,self.t][4:6].copy()
        
        cost = self.w[0]*np.round(np.linalg.norm(hip - u), 2)
        if self.isdone():
            cost += 100
        cost += self.w[1]*np.linalg.norm(self.sim_data[:,self.t][2:4])
        if np.round(self.sim_data[:,self.t][4] - self.sim_data[:,self.t - 5][4], 2) != 0 or \
            abs(np.round(self.sim_data[:,self.t][5] - self.sim_data[:,self.t - 5][5], 2)) != self.b:
            cost += self.w[2]
            
        return cost
    
    def random_action(self):
        '''
        Genarates random action
        '''
        action_x = np.random.randint(len(self.action_space_x))
        action_y = np.random.randint(len(self.action_space_y))
        
        return np.array([action_x, action_y])

    def show_episode(self, freq):
        '''
        Shows animation
        Input :
            freq : frame rate
        '''
        sim_data = self.sim_data[:,::freq]
        
        fig = plt.figure()
        ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1))
        text_str = "top view of 2d LIPM"
        base, = ax.plot([], [], lw=3, color = 'blue')
        leg, = ax.plot([], [], lw=3, color = 'pink')
        com, = ax.plot([], [], 'o', color='red')
        foot, = ax.plot([], [], 'o', color='green')

        
        def init():
            base.set_data([], [])
            leg.set_data([], [])
            com.set_data([], [])
            foot.set_data([], [])
            return base, leg, com, foot
        
        def animate(i):
            x_com = sim_data[:,i][0]
            y_com = sim_data[:,i][1]
            u_x = sim_data[:,i][4]
            u_y = sim_data[:,i][5]
            n = sim_data[:,i][7]
            
            base.set_data([x_com, x_com], [y_com - self.b/2, y_com + self.b/2])
            leg.set_data([x_com, u_x], [y_com - n*(self.b/2), u_y])
            com.set_data([x_com], [y_com])
            foot.set_data([u_x], [u_y])
            return base,leg, com, foot
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)
        plt.grid()
        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))

In [191]:
show = False

if show:
    no_steps = 5 ## simulates 10 steps taken by the inverted pendulum
    step_time = 0.1 ## duration after which step is taken by pendulum

    env = TwoDLipmEnv(0.2, 0.16, 0.22, [1, 0, 0], 9)
    state = env.reset_env([0.0,0.0, 1.0, 0.0], no_steps*step_time)
    for t in range(no_steps):
    #     action = env.random_action()
        if np.power(-1, t) > 0:
            action = [7, 4]
        elif np.power(-1, t) < 0:
            action = [7, 4]

        next_state, cost, done = env.step_env(action, step_time)
        print(state, action, cost, next_state, done)
        state = next_state

    env.show_episode(5)


[0. 0. 1. 0. 1.] [7, 4] 0.07 [-0.05 -0.13  1.25  0.42 -1.  ] False
[-0.05 -0.13  1.25  0.42 -1.  ] [7, 4] 0.1 [-0.09  0.11  1.29 -0.16  1.  ] False
[-0.09  0.11  1.29 -0.16  1.  ] [7, 4] 0.13 [-0.13 -0.11  1.14  0.4  -1.  ] False
[-0.13 -0.11  1.14  0.4  -1.  ] [7, 4] 100.21 [-0.2   0.14  0.73 -0.05  1.  ] True
[-0.2   0.14  0.73 -0.05  1.  ] [7, 4] 100.33 [-0.33 -0.06 -0.16  0.69 -1.  ] True


<source type="video/mp4" src="data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAAx1G1kYXQAAAKuBgX//6rcRem9
5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIHIyODU0IGU5YTU5MDMgLSBILjI2NC9NUEVHLTQg
QVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE3IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv
eDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9
MHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm
PTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6
b25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9OSBsb29r
YWhlYWRfdGhyZWFkcz0xIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFj
ZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJh
bWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdl
aWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjUgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVz
aD0wIHJjX2xvb2thaGVhZD00MCByYz1jcmYgbWJ0cmVlPTEgY3JmPTIzLjAgcWNvbXA9MC42MCBx
cG1pbj0wIHFwbWF4PTY5IHFwc3RlcD00IGlwX3JhdGlvPTEuNDAgYXE9MToxLjAwAIAAABY6ZYiE
AC///vau/MsrRwuVLh1Ze7NR8uhJcv2IMH1oAAADAADXFXgkyfqBSIEjtBqXHwUhmp+Yw6LqLINi
2Q86JopuIgzVCEj0j6uci5RNrHO6iXCXNThYiLq8CBsa1ckLU+KaNHAZv/elULQAAAMA1L4a+msa
e35E0PP+ML7F2DEMByaVtegQGWG7L9MY2H3JOIN/3eceXEUqOUWkS5XGpbum34fksEDDDYS05vn7
fnWlbf7JI8qKrgu2ZZivBm0i+peXFJsLxmdb/+yeUgk5WDb0pbub6HIfRplbUMd+GHezE9KGTc8u
4Lqlq0boA10n0+9KBIS/+ltLhyBc+wn2Ik9vGlCQDmyeoW0NYhWsBvFjH/juZKLvUMG8Ojzybs2h
8oZdhbyFTPWs9o4/f9uqKUMrpK+phFsIELvY1HAxGbTKBKqD0XScZoHOPHGywgB+siNZ+Vo/JX08
MMwJE0r1sp4DlCVZd41YjPQJjY5yrxBF8NZuBOK7qxvHOfDyookCn13Brz8ZAZ7g4ZkV6xBluL4d
zkQeKQxjqWtM57niZVjNG3WJYSF27ejTRO66PvzG4cPk/f9tOcbkRbfyjM+DDTiWaO+TjD5C45ds
cbI9qwSl9kWdK9Hz0emOOSWlZx/ahmrnIXCT0lDEvHQEI8Pjh1Coh0OpDBkkNBtCzSQnDGcHz+4c
w+YY9Aa9a3AU7cSu5JiW68IeXlbD9CCveTWWFnAa8o1nr/y/yoqpOCYyhqzm3YceJo80jSGLTbRb
mVDTRUoWW+ifngEUc5w782x8rT08WSvC9qxWVZ10Ytg6YQu+4j5vQ0oZNiw5FTfcZ0eeR2r/kOy4
MFFc91Snsx9kXMO+RVM36RM3oTSrmgYmYQFeE4bJ2LkHTt0mqPtBdTDq/vAbK8qkdiCCMnauv122
9FfDiqg13NstfSWI5KqeGyILGBAvc2BDoUdfjBifTymcUInzVKCCdBOpVvfVhqm4MehYRLpghaML
uxtT3AdfrNh0gdWMiVjF/VBqrZN8Eh4sIIZmIfiYnt/aYuBaskDLo1k2RezA8wz4+K/mbtn/+vbc
vt2Ivy6w3Rez5t9Gt3i5/1nfF8vhL4rhu8Zp4LYE4k9WHkm246CQK4+cuCSRLPfO4lXEhtKRm9xg
Q0PQfXn89fM/7FPMy+3jLwVeHwTg4ounblwQq7feP+zjC+RlKVXoCv83Xe8awUMjRDpw6nDcHkSO
Hj74+qcWlwSCmek9hToBr2n3COUydlFI9xEESCHq89I1IY3tR+3vAXkf2/zv995mHqZQXl3azE/+
NdE3hHdDM/g2wavnsRN7BqGOB8KlrTWX2KhPs4a05Su8qHkl15FFgloQjjLcqIqjKfDIw5UhnNNY
wCTqm8VW/Pwt8GjR0+cGUXBo1KAXrmZc36Mv9luhbzaak6/jJiIyPgEOaFS0eO5lVbppOILl1OpH
aB5g/HyjslgH4uV1COpgdomwjOjigK69pN244/DSKBAlV2SG1sMv81dlDr3NQndDPvssuZJb7G19
Yvdp2hXvxjEkx4nV+l9Ijub8J1fqNh/A/7whWtLq8b74wILAY/Oo9laDOVf2OQe7dlkneM18aTbz
mMvlHSjYGGw6IhPjZZbfa4RHAkEA9/gTlmRwp1C0YWjNDoGzegZn+zl0nKeIIhicH2Kj71vZERet
FHyv0iNVXQHB9GfEovNhI/BbMm2DOpm5/45FdrTx1mmaMya9+WcQthTngrWXqZFARsIEyrQC9LgZ
T2/iFUlyt2EBu3ypbPxnJoyxOBO6cT7tWqPQVTZz8nkI5KH/7++Q8PbIJhB4TjLjDPqvdukDjCGP
uSV+AWht3baB2zFYp6qyZmttv8OMwtpwDtILJwyozdzk+rfQkKxWjslyIqjcpScp4xcDlfVqjRB7
FRB0xzWnqD99LB2GXNGDARDS+Jx7mqf1D7i5UQbR+4pklANC21YFvked2zwV36wsTrWONF6AITAl
FO5uDw/KyXs1mfcujHpP7tZnn3R3gqlWbqPOK33YbKrnvMxVdy1Mp3ofs+bK6VunpriVngWx7dSl
qPp7YPU/TEKN9+xi7L+S6AZRY9/qS7L2w1KdfBsOGNmOq66hlEQo5lDmjrf4fWpPZuRufissjH/5
QeMSKpjg6mDZKP/jpqmESyXHDkdg4ly9JpRbktjblljf1FP/YLVTSjVplV86mspWBx6UGNX6mWXB
iVnp88A+9zkBNA/fYEzYZOs64oj1D7CJrwOXVc5NGLbmFX8BDU+WGtbyWR8xrWE/dE8f6tHA02N9
Nm14t/2r9s6RdyDvqFEKi9uwj+rZxE/cp78cgrotRv4E4AJgYbuCQSl6qTDCvqx1g1raeYbJgR9S
2XvO3t/SBNUCVGXitZwMP/+tSuyu4ohMY8gLFY4E6wPyUMacaUvLtoRYEbN/AORdK9i3BPakYyKW
dfDCpk+ruvWJ2kayRF/YYk+/Zzdi7uMxV/R3IaYnqZvatEKsN1vZ+8GfwkRRakRFeWJEZgoLrkEk
Go8t/p5MIli2sIzGwgNM3PcfmjTs//RZJn1NVCOCs3VDu9IXz3vgiPCfnfgLYaGo2V01ZSXtA8KR
4tGtsUMSUlmROAqZMfVXsv0Me8vs57a9LkNj8XdmEp/qN1ES/hi3chAfBbl5UxQBVlCPZsNhGhyh
BJU9Yz2QAo+Cicy2L/4nyYkaCQWF7Jgj3TbvbOm+0HOXAuitKc0TDNP+onpkUqx+A7ZQIkzHv/t2
0MbSJd3CJNN7lQia1FtfMlO7cnRFo7JhASD4Hg7w5XCTLOFvm444htiZf4sI0XoDZkABetInwaKS
4N/+yZfOM7LeICTmlV/8MZ3BEYeah/UbRgJX69wy3lq4Ea5CpijRZnv5WFwlyDKqTdQsBs3J7yQA
3zNZcer4XdpoxryR1jAUb